In [ ]:
import pandas as pd
import numpy as np
import missingno as mno
from sklearn import linear_model
from tqdm import tqdm
from pathlib import Path
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import xgboost
import catboost as cb

EDA can be found in my other [notebook](https://www.kaggle.com/code/robertturro/eda-feature-distributions-skewness-and-kurtosis)

In [ ]:
input_path = Path('/kaggle/input/tabular-playground-series-jun-2022/')

data = pd.read_csv(input_path / 'data.csv', index_col='row_id')
submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='row-col')

In [ ]:
cols = data.columns

In [ ]:
#imptr = IterativeImputer(
#    estimator=xgboost.XGBRegressor(
#        n_estimators=100,
#        random_state=42,
#        tree_method='gpu_hist',
#    ),
#    missing_values=np.nan,
#    max_iter=10,
#    initial_strategy='mean',
 #   imputation_order='ascending',
 #   verbose=2,
#    random_state=42
# )


imptr = IterativeImputer(
    estimator=cb.CatBoostRegressor(
        n_estimators=100,
        task_type='GPU'),
    missing_values=np.nan,
    max_iter=10,
    initial_strategy='mean',
    imputation_order='ascending',
    verbose=2,
    random_state=42
)

imp_data = imptr.fit_transform(data)
data = pd.DataFrame(data=imp_data, columns=cols)

In [ ]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = data.loc[row, col]

submission.to_csv('submission.csv')